In [1]:
from utilities import *
from core.initial import Insertions
from core.solver import CGCPSolver
from docplex.cp.model import *
from docplex.mp.model import Model
import pandas as pd

In [2]:
datafile = "D:/Google Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"
d = DataProvider(filepath=datafile, route='910')
model = CSPModel(d)
model.build_model()
initial = Insertions(model)
initial.solve()
s = CGCPSolver(initial.sol)

In [ ]:
rmp = Model(name='Restricted_Master_Problem')

rmp.trips = s.trips
rmp.duties = s.duties

rmp.trip_duty_arr = rmp.binary_var_matrix(s.trips, s.duties)

In [ ]:
model.data

In [ ]:
rmp.duty_arr = rmp.binary_var_list(rmp.duties, name='duty')

In [ ]:
for trip in rmp.trips:
    duty_ct = rmp.sum(rmp.trip_duty_arr[(trip, duty)] * rmp.duty_arr[duty.ID] for duty in rmp.duties) == 1
    rmp.add_constraint(duty_ct)

In [ ]:
rmp.minimize(rmp.sum(rmp.duty_arr[duty.ID] for duty in rmp.duties))

In [ ]:
rmp.solve()

In [51]:
sub = CpoModel(name="Pricing_Subproblem")

In [29]:
ntrips = len(model.trips)
nduties = len(model.trips)

In [ ]:
trips = [interval_var(start=trip.start_time,
                      end=trip.end_time,
                      size=trip.duration,
                      name=f'Trip_{idx}') for idx, trip in enumerate(model.trips)]


In [ ]:
model.start_time_arr

In [52]:
duties = [interval_var(start=(model.data['start_time'].min(), model.data['start_time'].max()),
                       end=(model.data['end_time'].min(),
                            model.data['end_time'].max()),
                       size=model.constraints.shift_span,
                       name=f"Duty_{i}",
                       optional=True) for i in range(nduties)]


In [53]:
trip2trip = integer_var_list(ntrips, min=0, max=ntrips + 1, name='Trip2Trip')
trip2duty = integer_var_list(ntrips, min=0, max=nduties, name='Trip2Duty')

In [54]:
start_time = [[integer_var(min=0, max=model.constraints.shift_span, name=f"StartTime-{i}-{j}") for j in range (nduties)] for i in range(ntrips)]

In [55]:
cdt = integer_var_list(nduties, min=0, max=model.constraints.continuous_driving, name="CDT")
tdt = integer_var_list(nduties, min=0, max=model.constraints.total_driving, name="TDT")

In [56]:
for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(trip2trip[i] == j, model.end_time_arr[i] <= model.start_time_arr[j]))

In [57]:
for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(trip2trip[i] == j, model.end_loc_arr[i] <= model.start_loc_arr[j]))

In [58]:
for i in range(ntrips):
    sub.add(trip2trip[i] != i)

In [59]:
for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(trip2trip[i] == j, trip2duty[i] == trip2duty[j]))

In [ ]:
for i in range(ntrips):
    sub.add(trip2trip[i] != 0)
    # sub.add(sum([trip2trip[j] == i for j in range(ntrips+1)]) == 1)

In [11]:
for i in range(ntrips-1):
    for j in range(i + 1, ntrips):
        sub.add(end_before_start(trips[i], trips[j]))

NameError: name 'trips' is not defined

In [ ]:
has_break = [[binary_var(name=f'Break_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]
has_rest = [[binary_var(name=f'Rest_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]

In [ ]:
trip_seq = sequence_var(trips)

In [60]:
msol = sub.solve()

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Satisfiability problem - 116 variables, 10150 constraints
 ! Initial process time : 0.06s (0.06s extraction + 0.01s propagation)
 !  . Log search space  : 600.2 (before), 600.2 (after)
 !  . Memory usage      : 2.5 MB (before), 2.5 MB (after)
 ! Using parallel search with 4 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 *                     69  0.09s        1         4  = Trip2Duty_13
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 272
 ! Number of fails        : 0
 ! Total memory usage     : 16.0 MB (16.0 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.09s (0.04s engine + 0.06s extraction)
 ! Search speed 

In [61]:
sduties = []
for i in range(ntrips):
    print(str(i)+" to "+str(msol[trip2trip[i]]) + ' - ' + str(msol[trip2duty[i]]))
    duties.append(str(msol[trip2duty[i]]))

print(set(sduties))

0 to 39 - 44
1 to 45 - 38
2 to 35 - 38
3 to 45 - 38
4 to 25 - 39
5 to 35 - 38
6 to 12 - 50
7 to 15 - 38
8 to 46 - 4
9 to 18 - 22
10 to 41 - 22
11 to 51 - 38
12 to 34 - 50
13 to 59 - 4
14 to 36 - 46
15 to 57 - 38
16 to 28 - 39
17 to 40 - 22
18 to 41 - 22
19 to 34 - 50
20 to 29 - 22
21 to 35 - 38
22 to 42 - 22
23 to 54 - 39
24 to 57 - 38
25 to 54 - 39
26 to 51 - 38
27 to 55 - 22
28 to 54 - 39
29 to 41 - 22
30 to 50 - 4
31 to 48 - 22
32 to 54 - 39
33 to 44 - 22
34 to 53 - 50
35 to 51 - 38
36 to 59 - 46
37 to 59 - 51
38 to 53 - 50
39 to 58 - 44
40 to 55 - 22
41 to 56 - 22
42 to 55 - 22
43 to 46 - 4
44 to 56 - 22
45 to 51 - 38
46 to 50 - 4
47 to 50 - 4
48 to 59 - 22
49 to 52 - 4
50 to 52 - 4
51 to 57 - 38
52 to 59 - 4
53 to 59 - 50
54 to 58 - 39
55 to 56 - 22
56 to 58 - 22
57 to 58 - 38
set()


In [ ]:
td_matrix = {}
for trip in model.trips:
    for duty in range(ntrips):
        td_matrix[(trip.ID, duty)] = binary_var(name=f"T{trip.ID}-D{duty}")

In [ ]:
end_of(trips[0]).equals(trips[1])

In [ ]:
INTERVAL_MAX